In [1]:
import warnings
warnings.filterwarnings('ignore')

from libsss import *
from model_optimizer import *
from tester import *

from model_opt import *


In [2]:
def accuracy_calculate(actual_values, predicted_values):
    comparison = abs(np.round(predicted_values) - actual_values)
    accuracy = 1- ((len(comparison[comparison>=(0+1)])) / len(actual_values))
    return accuracy

def mean_iou_calculator(actual_values, predicted_values, time):
    confusion_array = confusion_matrix(actual_values, predicted_values)
    individual_ious = []
    for i in range(len(confusion_array)):
        individual_iou = confusion_array[i][i] / (sum(confusion_array[i]))
        individual_ious.append(individual_iou)
    mean_iou = sum(individual_ious)/len(individual_ious)
    results = pd.DataFrame()  
    featue_y_values = sorted(data[time].unique())
    for i in range(len(individual_ious)):
        results.insert(0, 'iou_(' + str(featue_y_values[i])  +')', [individual_ious[i]], True)
    results = results[results.columns[::-1]]
    results.insert(0, 'mean_iou', mean_iou, True)
    return results


########################################################################################### READ CSV 
def read(csv):
    data = pd.read_csv(csv, sep=';', encoding = "utf8")
    #Rearannge the dataframe as the old one
    data.columns = ['product', 'amount', 'company', 'town', 'type', 'order date', 'delivery date', 'time']

    data2 = data.drop('delivery date', axis = 1)
    data2["order day"] = ''
    data2["order month"] = ''
    data2["week day"] = ''

    # Remove 'order date' and add 'order day', 'order month' and 'week day' features
    for i in range(len(data2)):
      data2.at[i, 'order day'] = data2['order date'][i].split()[0]
      data2.at[i, 'order month'] = data2['order date'][i].split()[1]
      data2.at[i, 'week day'] = data2['order date'][i].split()[-1]
    data2 = data2.drop('order date', axis = 1)
    data2['week day'] = data2['week day'].str.replace('Pazartesi','pts')
    data2['week day'] = data2['week day'].str.replace('Salı','sal')
    data2['week day'] = data2['week day'].str.replace('Çarşamba','çrş')
    data2['week day'] = data2['week day'].str.replace('Perşembe','prş')
    data2['week day'] = data2['week day'].str.replace('Cumartesi','cts')
    data2['week day'] = data2['week day'].str.replace('Cuma','cum')
    data2['week day'] = data2['week day'].str.replace('Pazar','paz')

    # data2 = data2[data2["week day"].str.contains("Pazar") == False]

    # Rearranging Dataframe
    data2 = data2[['product', 'company', 'amount', 'town', 'type', 'order day', 'week day', 'order month', 'time']]
    data2['town'] = data2['town'].str.lower()
    data2['town'] = data2['town'].str.replace('i̇','i')
    data2['town'] = data2['town'].str.replace('.','missing')
    data2['town'] = data2['town'].str.replace(' tekirdağ','tekirdağ')
    data2['town'] = data2['town'].str.replace('küçükçekmece','istanbul')
    data2['town'] = data2['town'].str.replace('çorlu','tekirdağ')
    data2['town'] = data2['town'].str.replace('bandirma','balıkesir')

    #data2 = data2.drop('town',axis = 1).reset_index(drop=True)
    data2 = data2.drop('order day',axis = 1).reset_index(drop=True)
    #data2 = data2.drop('order month',axis = 1).reset_index(drop=True)
    data2 = data2.fillna("missing")
    #data2 = data2[data2["town"].str.contains("missing") == False]

    data_clean = data2.copy()
    drop_df = data2.copy()
    drop_index_list = []
    return data2, data_clean ,drop_df, drop_index_list

data2, data_clean ,drop_df, drop_index_list = read('gulle.csv')


########################################################################################### REMOVE INSUFFICIENT DATA
def remove_insuff(df, ft):
    print("removing insufficient data for", ft, "...")
    fst_len = len(df)
    x = df[ft].value_counts() < 10
    df2 = x.to_frame().reset_index()
    df2.columns = [ft, 'booly']
    df2.drop(df2[df2.booly == False].index, inplace=True)
    drop_list = df2[ft].tolist()
    drop_indices=[]

    if len(drop_list) != 0:
        for i in df.index:
            for j in range(len(drop_list)):
                if (drop_list[j] == df.at[i, ft]):
                    drop_indices = drop_indices + [i]
        df.drop(drop_indices, inplace=True)
        
    else:
        drop_indices = []
                        
    lst_len = len(df)
    rem = fst_len - lst_len      # number of removed data
    per = (rem / fst_len) * 100  # percentage of removed data

    print("total number of removed data: ", rem)
    print("persentage of removed data: ", round(per, 2), "%")
    return df, drop_indices

data2, drop_indices = remove_insuff(data2, "company")
drop_index_list += drop_indices
print("\n")

data2, drop_indices = remove_insuff(data2, "product")
drop_index_list += drop_indices


########################################################################################### CLEAN NOISE
print("\n")
def clean_noise(df): 
    in_len = len(df)
    zs = 0.89
    
    n_prod = df["product"].nunique()
    prod_list = df["product"].unique()
    n_comp = df["company"].nunique()
    comp_list = df["company"].unique()
    
    print("Cleaning noise ... ")
    
    index_drop_list = []
    for prod in prod_list:

        df_max_scaled = df[df["product"] == prod].copy()

        for comp in comp_list:
            df_max_scaled2 = df_max_scaled[df_max_scaled["company"] == comp].copy()

            if len(df_max_scaled2) > 1:
                
                max_min_t = df_max_scaled2["time"].max() - df_max_scaled2["time"].min()
                max_min_a = df_max_scaled2["amount"].max() - df_max_scaled2["amount"].min()
                
                if (max_min_a != 0) and (max_min_t != 0):
                    df_max_scaled2["time"] = (df_max_scaled2["time"] - df_max_scaled2["time"].min()) / max_min_t
                    t_sc = df_max_scaled2[["time"]]
                    df_zscore_t = (t_sc - t_sc.mean())/t_sc.std()
                    dfz_t = abs(df_zscore_t) > zs

                    df_max_scaled2["amount"] = (df_max_scaled2["amount"] - df_max_scaled2["amount"].min()) / max_min_a
                    amo_sc = df_max_scaled2["amount"]
                    df_zscore_a = (amo_sc - amo_sc.mean())/amo_sc.std()
                    dfz_a = abs(df_zscore_a) > zs

                    df1 = dfz_t["time"] & dfz_a 
                    df2 = (df_zscore_t["time"] * df_zscore_a) < 0 
                    dfz = df1 & df2 

                    index_drop_list = index_drop_list + [*filter(dfz.get, dfz.index)]

    index_drop_list = sorted(list(set(index_drop_list)))
    df.drop(index_drop_list, axis=0, inplace=True)
    rem = len(index_drop_list)
    f_len = len(df)
    n_del = in_len - f_len
    per = (n_del / in_len) * 100
    
    #print("deleted indices: ",index_drop_list)
    print("total number of removed data: ", n_del)
    print("persentage of removed data: ", round(per, 2), "%")
    
    return df, index_drop_list

def clean_noise_tt(df): 
    in_len = len(df)
    zst = 1.5
    
    n_prod = df["product"].nunique()
    prod_list = df["product"].unique()
    n_comp = df["company"].nunique()
    comp_list = df["company"].unique()
        
    print("Cleaning time noise ... ")
    
    index_drop_list = []
    for prod in prod_list:

        df_max_scaled = df[df["product"] == prod].copy()

        for comp in comp_list:
            df_max_scaled2 = df_max_scaled[df_max_scaled["company"] == comp].copy()

            if len(df_max_scaled2) > 1:
                
                max_min_t = df_max_scaled2["time"].max() - df_max_scaled2["time"].min()
                
                if (max_min_t != 0):
                    df_max_scaled2["time"] = (df_max_scaled2["time"] - df_max_scaled2["time"].min()) / max_min_t
                    t_sc = df_max_scaled2[["time"]]
                    df_zscore_t = (t_sc - t_sc.mean())/t_sc.std()
                    dfz_t = abs(df_zscore_t) > zst
                    
                    index_drop_list += dfz_t[dfz_t["time"].eq(True)].index.tolist()
                    
                    #index_drop_list = index_drop_list + [*filter(dfz_t.get, dfz_t.index)]

    index_drop_list = sorted(list(set(index_drop_list)))
    df.drop(index_drop_list, axis=0, inplace=True)
    rem = len(index_drop_list)
    f_len = len(df)
    n_del = in_len - f_len
    per = (n_del / in_len) * 100
    
    #print("deleted indices: ",index_drop_list)
    print("total number of removed data: ", n_del)
    print("persentage of removed data: ", round(per, 2), "%")
    
    return df, index_drop_list

pd.options.mode.chained_assignment = None

data2, train_drop_list = clean_noise(data2)
drop_index_list = drop_index_list + train_drop_list
print("\n")

data2, train_drop_list = clean_noise_tt(data2)
drop_index_list = drop_index_list + train_drop_list

print("\n", "values: ")
print(data2.groupby('time').size())


########################################################################################### GROUP TIME
def group_time(df):
    min_samp = len(df) * 0.07
    t = 1
    gs = 0
    group = []
    sub_group = [t]
    #gap = df["time"].std()

    while t <= df["time"].max():
        
        
        if sum(df["time"] == t) > 0:
        
            gs += sum(df["time"] == t)

            if (len(sub_group) > 0) and ((t - min(sub_group)) <= math.ceil(math.sqrt(min(sub_group)))):
                sub_group += [t]

            else:
                if (len(sub_group) != 0):
                    gs = 0
                    group += [sub_group]
                sub_group = [t]

            if (gs >= min_samp) or ((t - min(sub_group)) > math.ceil(math.sqrt(min(sub_group)))):
                gs = 0
                group += [sub_group]
                sub_group = []
                
        if t == df["time"].max() :
            if len(sub_group) != 0:
                group += [sub_group]
            group[0].remove(1)
            
        t += 1
        
    return group

groups = group_time(data2)


########################################################################################### REMOVE INSUFFICIENT GROUPS
def remove_insuff_groups(df, groups):
    
    h = ()
    for i in range(len(groups)):
        a = 0
        for j in range(len(groups[i])):
            a += sum(df["time"] == groups[i][j])
        h += (a,)
    h = max(h)
    print("\h: ", h)
    
    
    c = True     # herhangi bir sub_groupta bulunan zamanlar dataframede 10 kereden az bulunuyorsa bu sub_groupu sil
    insuff_time = []
    while c:
        for i in range(len(groups)):
            s = 0
            for j in range(len(groups[i])):
                s += sum(df["time"] == groups[i][j])
            if s < (h * 0.05):           #----------------------------->>>>> fix
                insuff_time += groups[i]
        c = False

    return insuff_time

insuff_time = remove_insuff_groups(data2, groups)


########################################################################################### MAP TIME
def map_time(df, groups, insuff_time):
    print("Mapping time")
    index_drop_list = []
    for i in df.index:
        for t in insuff_time:
            if df.at[i, "time"] == t:
                index_drop_list += [i]
                
    df.drop(index_drop_list, axis=0, inplace=True)

    time = -1
    for sub_group in groups:
        
        time += 1
        for t in sub_group:
            for i in df.index:
                if df.at[i, "time"] == t:
                    df.at[i, "time"] = time
                    
        if sub_group[0] not in insuff_time:
            print( sub_group, " --------> ", time)

    return df

print("\n")
data2 = map_time(data2, groups, insuff_time)
print("insuff_time: ", insuff_time)
print("classes: ", sorted(data2["time"].unique()))



########################################################################################### ONE HOT ENCODING
def one_hot(df, ft):      ### ft = "company", "product", "week day" etc.
    print("one hot encoding ", ft, "...")
    dum = pd.get_dummies(df[ft])
    df = df.drop(ft, axis = 1)
    df = df.join(dum)
    print(ft, "encoded.")
    return df

print("\n")
data2  = one_hot(data2, "week day")
data2  = one_hot(data2, "company")
data2  = one_hot(data2, "product")
data2  = one_hot(data2, "order month")


########################################################################################### DROP_DF & DATA_CLEAN
drop_index_list = sorted(list(set(drop_index_list)))
drop_df = drop_df.loc[drop_index_list]

data_clean.drop(drop_df.index.to_list(), axis=0, inplace=True)


########################################################################################### DROP TOWN COLUMN
data2 = data2.drop('town',axis = 1).reset_index(drop=True)


########################################################################################### TRAIN-TEST SPLIT
from sklearn.model_selection import train_test_split

X = data2.copy()
Y = data2.copy()
X.drop("time", axis=1, inplace=True)
Y = Y["time"]

x_train, x_val, y_train, y_val = train_test_split(X,Y, test_size = 0.3, random_state = 42)


########################################################################################### NORMALIZE AMOUNT
xt_min = x_train["amount"].min()
xt_max = x_train["amount"].max()

x_train["amount"] = (x_train["amount"] - xt_min) / (xt_max - xt_min)
x_val["amount"] = (x_val["amount"] - xt_min) / (xt_max - xt_min)


########################################################################################### DF_EMPTY
df_empty = x_train[0:0]


########################################################################################### PCA
def do_pca(x_train, x_val):
    from sklearn.decomposition import PCA
    xl = len(x_train.columns)
    pca = PCA(.95)
    pca.fit(x_train)
    print("number of features dropped from ", xl, " to ", pca.n_components_) 
    #print("variance ratio: ", pca.explained_variance_ratio_) 

    x_train = pca.transform(x_train)
    x_val = pca.transform(x_val)
    return pca, x_train, x_val

print("\n")
pca, x_train, x_val = do_pca(x_train, x_val)

print("\ndone...")


removing insufficient data for company ...
total number of removed data:  73
persentage of removed data:  6.54 %


removing insufficient data for product ...
total number of removed data:  293
persentage of removed data:  28.09 %


Cleaning noise ... 
total number of removed data:  18
persentage of removed data:  2.4 %


Cleaning time noise ... 
total number of removed data:  56
persentage of removed data:  7.65 %

 values: 
time
1     395
2     106
3      80
4      46
5      18
6      14
7       5
9       3
10      2
11      2
12      1
14      2
34      1
44      1
dtype: int64
\h:  395


Mapping time
[1]  -------->  0
[2]  -------->  1
[3]  -------->  2
[4, 5]  -------->  3
[6, 7, 9]  -------->  4
insuff_time:  [10, 11, 12, 14, 34, 44]
classes:  [0, 1, 2, 3, 4]


one hot encoding  week day ...
week day encoded.
one hot encoding  company ...
company encoded.
one hot encoding  product ...
product encoded.
one hot encoding  order month ...
order month encoded.


number of features drop

In [3]:
# optimizerr(x_train, y_train, x_val, y_val)

In [ ]:
LR_best, XGB_best, RF_best, KN_best, MLP_best = opt_params(x_train, y_train, x_val, y_val, 1000)

Optimizing LogisticRegression Parameters...
100%|████████████████████████████████████████████| 1000/1000 [01:38<00:00, 10.12trial/s, best loss: 3.6416506697112685]

Optimizing XGBClassifier Parameters...
  8%|███▌                                          | 77/1000 [06:33<1:43:38,  6.74s/trial, best loss: 2.73733708212279]

In [ ]:
LR_opt, XGB_opt, RF_opt, KN_opt, MLP_opt = get_results(LR_best, XGB_best, RF_best, KN_best, MLP_best, x_train, y_train, x_val, y_val)

# Take input and create df_inp 

In [7]:
def is_prod_in_data(df):
    prods = df["product"].unique()
    prd = str(input("Product seç: "))
    if prd not in prods: 
        return False, prd
    else:
        return True, prd
    
def is_prod_in_data_drop(df, prd):
    prods = df["product"].unique()
    if prd not in prods: 
        return False
    else:
        return True
    
def take_input(df, prd):
    amo = input("Amount: ") 
    y = False
    while y == False:
        if (amo.isnumeric() == False):
            print("Pozitif tam sayı değer giriniz")
            amo = input("Amount: ")

        else:
            d = max(df[df["product"] == prd]["amount"].to_list()) * 3
            if int(amo) > d:
                print("Amount yüksek abi emin misin bak !?")
                y_n = input("y / n ?")
                if y_n == "y":
                    y = True
                elif y_n == "n":
                    amo = input("Amount: ")                   
            else:
                y = True
    amo = int(amo)

    w_days = ['pts', 'sal', 'çrş', 'prş', 'cum', 'cts', 'paz']
    wd = str(input("Week day: "))
    z = False
    while z == False:
        if wd not in w_days:
            print("Geçerli gün giriniz...")
            print("Geçerli günler: ", w_days)
            wd = str(input("Week day: "))
        else:
            z = True
            
            
    months = ['Ocak', 'Şubat', 'Mart', 'Nisan', 'Mayıs', 'Haziran', 'Temmuz', 'Ağustos', 'Eylül', 'Ekim', 'Kasım', 'Aralık']
    month = str(input("Ay: "))
    z = False
    while z == False:
        if month not in months:
            print("Geçerli ay giriniz...")
            print("Geçerli aylar: ", months)
            month = str(input("Ay: "))
        else:
            z = True

    typ = df[df["product"] == prd]["type"].unique()[0] 
    comps = df[df["product"] == prd]["company"].unique()
    tws = df[df["product"] == prd]["town"].unique()

    df_inp = pd.DataFrame(columns = ['product', 'company', 'amount', 'town', 'type', 'week day', 'order month'])
    
    for comp in comps:
        tw = df[(df["product"] == prd) & (df["company"] == comp)]["town"].unique()[0]

        df_inp = df_inp.append({'product' : prd, 'company' : comp, 'order month': month, 'amount' : amo, 
                                'town' : tw,'type' : typ, 'week day' : wd}, ignore_index=True)

    return df_inp, comps

def run():
    cond1 = False
    while cond1 == False:
        tf, prd = is_prod_in_data(data_clean)
        
        if tf:
            df_inp, comps = take_input(data_clean, prd)
            c = False
            cond1 = True
        else:
            if (is_prod_in_data_drop(drop_df, prd) == True):
                print("Güvenilir sonuç için product'a ait en az 5 giriş bulunmalıdır.")
                print("\n","Daha önce bu product alımları: ")
                a = drop_df[drop_df["product"] == prd]
                a = a.index.to_list()
                a = data.loc[a]
                df_inp = display(a[['company', 'amount', 'town', 'order date', 'delivery date', 'time']].style.hide(axis='index'))
                print ("Ortalama miktar = ", a["amount"].mean(), "Ortalama süre = ", a["time"].mean())
                comps = a["company"].unique()
                return df_inp, cond1, comps
    
            else:
                df_inp = print("Product bulunamadı.")

    if cond1:
        df_inp  = one_hot(df_inp, "week day")
        df_inp  = one_hot(df_inp, "product")
        df_inp  = one_hot(df_inp, "company")
        df_inp  = one_hot(df_inp, "order month")

        df_inp = df_inp.drop('town',axis = 1).reset_index(drop=True)

        df_inp["amount"] = (df_inp["amount"] - xt_min) / (xt_max - xt_min)

        df_inp = df_empty.append(df_inp)   # for the next version of pandas use the next line of code instead of this one. 
        #df_inp = pd.concat([df_inp, df_empty])   ------->>> for the future version of pandas
        
        df_inp = df_inp.fillna(0)
        df_inp = pca.transform(df_inp)
        
    return df_inp, cond1, comps
        

# Predict

In [8]:
df_inp, cond1, comps = run()

if cond1:
    LR = LR_opt.predict(df_inp)
    RF = RF_opt.predict(df_inp)
    KN = KN_opt.predict(df_inp)
    XGB = XGB_opt.predict(df_inp)
   
    print("LR: ", LR, "\nRF:", RF, "\nKN:", KN, "\nXGB:", XGB)
    
    res = np.array([])
    output = np.array([])
    for i in range(len(LR)):
        res = np.append(res, [LR[i], RF[i], KN[i], XGB[i]])
        m = mode(res)[0][0]
        output = np.append(output, m)
        res = np.array([])
    
        print("for company ", comps[i], " predicted time = ", output[i])
        
del(df_inp)

Product seç: K730
Amount: 444
Week day: sal
Ay: OCak
Geçerli ay giriniz...
Geçerli aylar:  ['Ocak', 'Şubat', 'Mart', 'Nisan', 'Mayıs', 'Haziran', 'Temmuz', 'Ağustos', 'Eylül', 'Ekim', 'Kasım', 'Aralık']
Ay: Ocak


,product,company,amount,town,type,week day,order month
0,K730,T-060,444,tekirdağ,1,sal,Ocak
1,K730,T-0142,444,missing,1,sal,Ocak


one hot encoding  week day ...
week day encoded.
one hot encoding  product ...
product encoded.
one hot encoding  company ...
company encoded.
one hot encoding  order month ...
order month encoded.
LR:  [1 0] 
RF: [0 0] 
KN: [0 1] 
XGB: [0 1]
for company  T-060  predicted time =  0.0
for company  T-0142  predicted time =  0.0
